In [1]:
import os 
import math
import googlemaps
from dotenv import load_dotenv

from pydantic import BaseModel

load_dotenv()

GOOGLE_MAPS_API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")
client = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)

In [2]:
class ETARequest(BaseModel):
    start: str
    destination: str

def calc_eta_sync(start: str, destination: str) -> int:
    """
    Synchronous function that calculates ETA using the Google Maps Distance Matrix API.
    Applies a 20% safety buffer and returns the ETA in minutes.
    """
    # params = {
    #     "origins": start,
    #     "destinations": destination,
    #     "key": GOOGLE_MAPS_API_KEY,
    #     "mode": "driving"
    # }

    response = client.distance_matrix(origins=start, destinations=destination, mode="driving")
    print(response)
    # The response is already a dictionary, no need to call json()
    data = response
    
    if data.get("status") != "OK":
        raise Exception("Error with Distance Matrix API: " + data.get("error_message", "Unknown error"))
    
    element = data["rows"][0]["elements"][0]
    if element.get("status") != "OK":
        raise Exception("No route found: " + element.get("status", "Unknown error"))
    
    duration_in_seconds = element["duration"]["value"]
    buffered_duration = duration_in_seconds * 1.2  # Apply 20% safety buffer
    eta_minutes = math.ceil(buffered_duration / 60)
    return eta_minutes

In [3]:
eta = calc_eta_sync("Grand Terrace, CA", "USC, LA, CA")

print(eta)

{'destination_addresses': ['Los Angeles, CA 90007, USA'], 'origin_addresses': ['Grand Terrace, CA, USA'], 'rows': [{'elements': [{'distance': {'text': '99.2 km', 'value': 99247}, 'duration': {'text': '1 hour 5 mins', 'value': 3883}, 'status': 'OK'}]}], 'status': 'OK'}
78
